$$
\huge \textsf{Gender gap between countries around the world}
$$

 Delving into data enrichment

##  Data preprocessing

The `data.csv` dataset we are using can be downloaded from [here](https://tcdata360.worldbank.org/indicators/af52ebe9?country=BRA&indicator=27959&viz=line_chart&years=2006,2018). We briefly describe its content. It contains 5586 samples and 17 columns, which describe:

- **Country ISO3**: string country code
- **Country Name**: string country name
- **Indicator Id**: integer identificator of each measured index
- **Indicator**: string name of the index
- **Subindicator Type**: string of the type of indicator ('Rank' or 'Index')
- **2006**,**2007**,**2008**,**2009**,**2010**,**2011**,**2012**,**2013**,**2014**,**2015**,**2016**,**2017**,**2018**: float score of each indicator in the corresponding year. The highest possible score is 1 (equality) and the lowest possible score is 0 (inequality).

Load the `data.csv` file and get all rows which contain, aside from the ***Overall Global Gender Gap Index***, four more indicators: the ***Global Gender Gap Political Enpowerment Subindex***, the ***Global Gender Gap Economic Participation and Opportunity Subindex***, the ***Global Gender Gap Educational Attainment Subindex*** and the ***Global Gender Gap Health and Survival Subindex***. All of them with the subindicator type 'Index', not 'Rank'.

After getting the *Country Name*, the *Indicator* and the score values by *years*, we can drop everything else. Notice that, in this case, we also drop those samples in which at least one or more atributes are missing, as in the final visualization we do not want to plot null values and, as we have plenty of data, doing that is not a issue. Moreover, by doing that we avoid possible errors when selecting rows in Altair.

In [ ]:
indices = pd.read_csv('data/data.csv', sep=';')
indices = indices.loc[
    (indices['Indicator'].isin(['Overall Global Gender Gap Index',
                            'Global Gender Gap Political Empowerment subindex',
                            'Global Gender Gap Economic Participation and Opportunity Subindex',
                            'Global Gender Gap Educational Attainment Subindex',
                            'Global Gender Gap Health and Survival Subindex'])) &
    (indices['Subindicator Type'] == 'Index')]
indices = indices.drop(['Country ISO3','Indicator Id','Subindicator Type','2018'], axis=1)
indices = indices.dropna()
print(indices.shape)
indices.head()

(535, 13)


,Country Name,Indicator,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
38,Albania,Overall Global Gender Gap Index,0.6607,0.6685,0.6591,0.6601,0.6726,0.6748,0.6655,0.6412,0.6869,0.701,0.704
40,Albania,Global Gender Gap Political Empowerment subindex,0.0375,0.0375,0.0413,0.0413,0.0785,0.0784,0.0753,0.0256,0.1834,0.214,0.214
43,Albania,Global Gender Gap Economic Participation and O...,0.6613,0.6885,0.6491,0.6532,0.6808,0.7129,0.6663,0.6324,0.6534,0.671,0.668
45,Albania,Global Gender Gap Educational Attainment Subindex,0.9888,0.9925,0.9907,0.9906,0.9942,0.9809,0.9890,0.9755,0.9700,0.972,0.986
47,Albania,Global Gender Gap Health and Survival Subindex,0.9553,0.9553,0.9553,0.9553,0.9370,0.9268,0.9313,0.9313,0.9409,0.947,0.947


The first difference between the previous visualization and this one is that, here, we take the long-form data approach beforehand by transforming the dataframe and we do not leave it to do with Altair. Broadly, using pandas *melt* function we combine all years columns into only two, representing the year and its corresponding score.

In [ ]:
indices = indices.melt(id_vars=['Country Name','Indicator'], var_name='year', value_name='score')
print(indices.shape)
indices.head()

(5885, 4)


,Country Name,Indicator,year,score
0,Albania,Overall Global Gender Gap Index,2006,0.6607
1,Albania,Global Gender Gap Political Empowerment subindex,2006,0.0375
2,Albania,Global Gender Gap Economic Participation and O...,2006,0.6613
3,Albania,Global Gender Gap Educational Attainment Subindex,2006,0.9888
4,Albania,Global Gender Gap Health and Survival Subindex,2006,0.9553


In [ ]:
indices['Country Name'].nunique()

107

At this point we have data about **107 countries**. It is important to keep checking this number when appending new datasets in order to ensure consistency.

Next, we load the first extra dataset used in this visualization.

The `gdp.csv` dataset we are using can be downloaded from [here](https://data.worldbank.org/indicator/NY.GDP.MKTP.CD?view=chart), provided by *The World Bank Group*. We briefly describe its content. It contains 263 samples and 13 columns, which describe:

- **Country Name**: string country name
- **Indicator Name**: string name of the indicator
- **2006**,**2007**,**2008**,**2009**,**2010**,**2011**,**2012**,**2013**,**2014**,**2015**,**2016**: integer number of the Gross Domestic Product in current US dollars.

We chose this variable (the GDP) as it is a general known indicator which differentiates very well the countries in an economic level and it is very useful when making assumptions. So, we load the **GDP  (current US$)** by country:

In [ ]:
gdp = pd.read_csv('data/gdp.csv', sep=';')
gdp = gdp.drop(['Indicator Name'], axis=1)
#print(gdp.dtypes)
print(gdp.shape)
gdp.head()

(263, 12)


,Country Name,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Aruba,2.404527e+14,2.583513e+14,2.708470e+14,2.463045e+14,2.351260e+14,2.498599e+14,2.471370e+14,2.502510e+14,2.553357e+14,2.579638e+14,2.523960e+14
1,Afghanistan,2.637337e+14,3.596935e+14,3.646605e+14,4.380760e+14,5.433030e+14,5.911623e+13,6.418720e+14,6.371650e+13,6.138563e+14,5.784664e+14,5.472281e+14
2,Angola,2.599566e+14,3.121996e+14,4.080941e+14,3.122781e+14,3.587884e+14,4.615468e+14,5.100096e+14,5.254882e+14,5.408410e+14,4.166980e+14,3.506073e+14
3,Albania,2.972743e+14,3.595037e+14,4.370540e+14,4.114140e+14,4.094362e+14,4.437178e+14,4.247614e+14,4.413082e+13,4.578667e+14,3.952829e+14,4.124109e+14
4,Andorra,4.374769e+14,4.858222e+14,4.778509e+13,4.333887e+14,3.973635e+14,4.110073e+14,3.839294e+14,4.062675e+14,4.230033e+14,3.603965e+14,3.722411e+14


We follow the same approach as above, that is, melting the year columns in two, representing the year and the corresponding GDP value. This way it will be easier to work with Altair.

In [ ]:
gdp = gdp.melt(id_vars=['Country Name'], var_name='year', value_name='gdp')
print(gdp.shape)
gdp.head()

(2893, 3)


,Country Name,year,gdp
0,Aruba,2006,2.404527e+14
1,Afghanistan,2006,2.637337e+14
2,Angola,2006,2.599566e+14
3,Albania,2006,2.972743e+14
4,Andorra,2006,4.374769e+14


In the same way, we load the second extra dataset used in this visualization.

The `pop.csv` dataset we are using can be download from [here](https://data.worldbank.org/indicator/SP.POP.TOTL), provided by *The World Bank Group*. We briefly describe its content. It contains 263 samples and 13 column, which describe:

- **Country Name**: string country name
- **Indicator Name**: string name of the indicator
- **2006**,**2007**,**2008**,**2009**,**2010**,**2011**,**2012**,**2013**,**2014**,**2015**,**2016**: integer number of the population of each country.

We chose this variable (the population) as it is a basic indicator which differentiates very well the countries in a social and economical level and it is very useful when making assumptions. So, we load the **population** by country:

In [ ]:
pop = pd.read_csv('data/pop.csv', sep=';')
pop = pop.drop(['Indicator Name'], axis=1)
print(pop.shape)
pop.head()

(263, 12)


,Country Name,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Aruba,100834.0,101222.0,101358.0,101455.0,101669.0,102046.0,102560.0,103159.0,103774.0,104341.0,104872.0
1,Afghanistan,26433049.0,27100536.0,27722276.0,28394813.0,29185507.0,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0
2,Angola,20149901.0,20905363.0,21695634.0,22514281.0,23356246.0,24220661.0,25107931.0,26015780.0,26941779.0,27884381.0,28842484.0
3,Albania,2992547.0,2970017.0,2947314.0,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0
4,Andorra,80993.0,82684.0,83862.0,84463.0,84449.0,83747.0,82427.0,80774.0,79213.0,78011.0,77297.0


Again, we follow the same approach we have been using, melting the years in two columns: year and population.

In [ ]:
pop = pop.melt(id_vars=['Country Name'], var_name='year', value_name='population')
print(pop.shape)
pop.head()

(2893, 3)


,Country Name,year,population
0,Aruba,2006,100834.0
1,Afghanistan,2006,26433049.0
2,Angola,2006,20149901.0
3,Albania,2006,2992547.0
4,Andorra,2006,80993.0


It should be stated we noted that as the data comes from the same source (i.e. *The World Bank Group*) and it is structured the same way as the `gdp.csv` dataset, both *Country Name* columns of the datasets are exactly the same. This means they have the same number of entries and those are sorted following the same order. This fact simplifies things a lot and eases the data enrichment process; a simple append of the columns is enough.

In [ ]:
both = gdp
both['population'] = pop['population']
print(both.shape)
both.head()

(2893, 4)


,Country Name,year,gdp,population
0,Aruba,2006,2.404527e+14,100834.0
1,Afghanistan,2006,2.637337e+14,26433049.0
2,Angola,2006,2.599566e+14,20149901.0
3,Albania,2006,2.972743e+14,2992547.0
4,Andorra,2006,4.374769e+14,80993.0


As a last resort, we load the third and last extra dataset used in this visualization.

The `regions.csv` data we are using can be download from [here](https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/blob/master/all/all.csv), a list provided by the GitHub user *lukes* (Luke Duncalfe). We briefly describe its content. It contains 249 samples and 11 columns, which describe:

- **name**: string country name
- **alpha-2**: string country code in 2-character form
- **alpha-3**: string country code in 3-character form
- **country-code**: integer country code
- **region**: string country region by continents ('Africa','Americas','Asia','Europe','Oceania')
- **sub-region**: string country sub-region by continents zones
- **intermediate-region**: string country intermediate-region detail of continent sub-region
- **region-code**: integer country region code
- **sub-region-code**: integer country sub-region code
- **intermediate-region-code**: integer country intermediate-region code
- **NULL**: void column with some random spare values

We chose this dataset in order to classify the countries list we have in continent regions. This way it will be easier to distinguish them for exemple by colors and quickly understand what role do the indicator variables play.

So, we load just the **name** and **region** dataframe:

In [ ]:
regions = pd.read_csv('data/regions.csv', sep=';')
regions = regions[['name','region']]
print(regions.shape)
regions.head()

(249, 2)


,name,region
0,Afghanistan,Asia
1,Åland Islands,Europe
2,Albania,Europe
3,Algeria,Africa
4,American Samoa,Oceania


Now we have all three `gdp.csv`, `pop.csv` and `regions.csv` datasets loaded we can start merging them to build our final dataframe.

On the one hand, we need to left-join `indices` (which contain all *Overall*, *Political*, *Educational*... index scores) with `regions` (continents) data by the country name, as we only want to preserve the countries for which we have infromation about the gender gap.

In [ ]:
with_reg = pd.merge(indices, regions, how='left', left_on=['Country Name'], right_on=['name'])
with_reg.head()

,Country Name,Indicator,year,score,name,region
0,Albania,Overall Global Gender Gap Index,2006,0.6607,Albania,Europe
1,Albania,Global Gender Gap Political Empowerment subindex,2006,0.0375,Albania,Europe
2,Albania,Global Gender Gap Economic Participation and O...,2006,0.6613,Albania,Europe
3,Albania,Global Gender Gap Educational Attainment Subindex,2006,0.9888,Albania,Europe
4,Albania,Global Gender Gap Health and Survival Subindex,2006,0.9553,Albania,Europe


On the other hand, we need to left-join the `with_reg` dataframe (i.e. index scores with country regions) we just created above with the `both` dataframe (which contains the *GDP* and *population* variables) by country name and year. Therefore, the final dataset  with which we will create the visualization is the following:

In [ ]:
alls = pd.merge(with_reg, both, how='left',
                   left_on=['Country Name','year'], right_on=['Country Name','year'])
alls['year'] = alls['year'].astype(int)
print(alls.shape)
alls.head()

(5885, 8)


,Country Name,Indicator,year,score,name,region,gdp,population
0,Albania,Overall Global Gender Gap Index,2006,0.6607,Albania,Europe,2.972743e+14,2992547.0
1,Albania,Global Gender Gap Political Empowerment subindex,2006,0.0375,Albania,Europe,2.972743e+14,2992547.0
2,Albania,Global Gender Gap Economic Participation and O...,2006,0.6613,Albania,Europe,2.972743e+14,2992547.0
3,Albania,Global Gender Gap Educational Attainment Subindex,2006,0.9888,Albania,Europe,2.972743e+14,2992547.0
4,Albania,Global Gender Gap Health and Survival Subindex,2006,0.9553,Albania,Europe,2.972743e+14,2992547.0


As proof that everything went well in the data enrichment process, we check there is no null value in the created dataset.

In [ ]:
alls[alls.isna().any(axis=1)]

,Country Name,Indicator,year,score,name,region,gdp,population


In addition, prior to the implementation, we save the sorted countries list for our future dropdown menus:

In [ ]:
countries_list = list(np.sort(alls['Country Name'].unique()))

Moreover, as the created dataframe have more than those 5000 rows that does Altair allows by default, we need to disable the max rows restriction:

In [ ]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Implementation

Finally, with all the data transformations performed and taking into account the nuances mentioned we can create the visualization. In the first place, we create a slider selection between years 2006 and 2016, starting at 2006.

In [ ]:
input_year = alt.binding_range(max=2016, min=2006, step=1)
yearSelect = alt.selection_single(fields=['year'], bind=input_year, name='Select', init={'year': 2006})

Next, we create an input dropdown for selecting the indicator to show, that is, whether select the *Overall*, *Political*, *Economic*, *Educational* or *Health* index. We set it to *Overall* by default.

In [ ]:
input_dropdown = alt.binding_select(
    options = ['Overall Global Gender Gap Index',
               'Global Gender Gap Political Empowerment subindex',
            'Global Gender Gap Economic Participation and Opportunity Subindex',
            'Global Gender Gap Educational Attainment Subindex',
            'Global Gender Gap Health and Survival Subindex'])
indicatorSelect = alt.selection_single(fields=['Indicator'], bind=input_dropdown, name='Choose', init={'Indicator':'Overall Global Gender Gap Index'})

Now, with these interactive input elements we can create our first chart. It is a **bubble chart** with the **x-axis fixed** with the **GDP  (current US$)** and with a **dynamic y-axis** showing the **score** of the selected dropdown index indicator and year (that last one picked throught a slider).

In addition, the **population** variable is encoded by the size of the bubble points and each one is colored with a different color depending on the **region** they belong to. Moreover, we add a tooltip showing all encoded information, i.e. the *Country Name*, *GDP*, *Index score*, *Population* and *Region*. It should be pointed that we have not been able to increase the minimum size of the bubble points as we did not found a proper way to change it with Altair.

Notice that aside from adding the two selectors (`yearSelect` and `indicatorSelect`) into the scene, we must apply a *Transform Filter* with an AND operator, which means we selected the row that matches the selected year column and the selected indicator column.

To provide a better look, we define a text chart which simply shows a big and clear textbox above the bubble chart, based on the dropdown index indicator selector.

In [ ]:
gdp_indices = alt.Chart(alls).mark_circle(opacity=1).encode(
    x=alt.X('gdp:Q', scale=alt.Scale(zero=False), title='GDP  (current US$)'),
    y=alt.Y('score:Q', scale=alt.Scale(zero=False)),
    size='population:Q',
    color='region:N',
    tooltip=['Country Name:N','gdp:Q','score:Q','population:Q','region:N']
).properties(
    width=600,
    height=300
).add_selection(
    yearSelect,
    indicatorSelect
).transform_filter(
    yearSelect & indicatorSelect
)

text = alt.Chart(alls).mark_text(dy=-170, dx=-65, size=20).encode(
    text='Indicator:N'
).transform_filter(
    indicatorSelect
)

Secondly, to ease the task of ranking countries based on these indicators, we want to display a **ranked bar chart** of the **top-20 countries by index**, over the period of time we have data.

The **y-axis** shows the **top-20 countries sorted by the score** of the selected index and year, and the **x-axis** shows the **score** itself. In addition, we provide a tooltip to precisely know the indicator score at a glance. The `yearSelect` selector is linked to the chart and, in combination with the `indicatorSelect`, a *Transform Filter* is applied, following the same strategy as before.

However, in order to achieve that ranked countries list, we make use of a *Transform Window*, which performs calculations over sorted groups of data objects, such as rankings. So, we sort the countries descending by its index score and we *Transform Filter* to show only those whith rank is equal to or above 20.

In [ ]:
bars = alt.Chart(alls).mark_bar().encode(
    x=alt.X('score:Q'),
    y=alt.Y('Country Name:N', sort=alt.SortField(field='score', order='descending')),
    color='region:N',
    tooltip=['score:Q']
).properties(
    title='Top-20 countries by Indicator score',
    width=200, height=300
).add_selection(
    yearSelect
).transform_filter(
    yearSelect & indicatorSelect
).transform_window(
    window=[{'op': 'rank', 'as': 'rank'}],
    sort=[{'field': 'score', 'order': 'descending'}]
).transform_filter(
    'datum.rank <= 20'
)

Finally, we create the **three-countries timeline comparison** of the previous indicators, over the same period of time. For doing so, we use a **line chart** where the **x-axis** shows the **11 years** and the **y-axis** shows the **indicator score**, this way the evolution of the selected index score can be easily seen.

Furthermore, we add three input dropdowns more for selecting three countries out of 107 from an alphabetical list, in order to compare their evolution for a selected index indicator over the 11 years we have data.

In [ ]:
countries_dropdown = alt.binding_select(options = countries_list)

countrySelect1 = alt.selection_single(fields=['Country Name'], bind=countries_dropdown, name='Pick1', init={'Country Name':'Spain'})
countrySelect2 = alt.selection_single(fields=['Country Name'], bind=countries_dropdown, name='Pick2', init={'Country Name':'Nigeria'})
countrySelect3 = alt.selection_single(fields=['Country Name'], bind=countries_dropdown, name='Pick3', init={'Country Name':'China'})

Each of the three plots are encoded with points to note that the available data is annual and lines simply join the points showing the trend between one year and another. Again, a tooltip is provided to lookup for some exact value. The data is filtered by follwing the same approach we have already used (adding the `indicatorSelect` and `countrySelect_i` to the scene and applying a *Transform Filter* with them).

In addition, to provide a better look, we define a text chart which simply shows a big and clear textbox above each line chart, based on the dropdown country selector. The chart for the first country looks like this:

In [ ]:
evol1 = alt.Chart(alls).mark_line(color='#2ca02c', point=True).encode(
    x=alt.X('year:N', scale=alt.Scale(zero=False), axis=alt.Axis(grid=True)),
    y=alt.Y('score:Q', scale=alt.Scale(zero=False)),
    tooltip=['Country Name:N','year:N','score:Q']
).add_selection(
    indicatorSelect,
    countrySelect1
).transform_filter(
    countrySelect1 & indicatorSelect
).properties(
    width=300,
    height=200
)

text1 = alt.Chart(alls).mark_text(dy=-120, size=18).encode(
    text='Country Name:N'
).transform_filter(
    countrySelect1 & indicatorSelect
)

The exactly same chart (but with a different line color) is used for the second country.

In [ ]:
evol2 = alt.Chart(alls).mark_line(color='#ff7f0e', point=True).encode(
    x=alt.X('year:N', scale=alt.Scale(zero=False), axis=alt.Axis(grid=True)),
    y=alt.Y('score:Q', scale=alt.Scale(zero=False)),
    tooltip=['Country Name:N','year:N','score:Q']
).add_selection(
    indicatorSelect,
    countrySelect2
).transform_filter(
    countrySelect2 & indicatorSelect
).properties(
    width=300,
    height=200
)

text2 = alt.Chart(alls).mark_text(dy=-120, size=18).encode(
    text='Country Name:N'
).transform_filter(
    countrySelect2 & indicatorSelect
)

And for the third and last country, the same.

In [ ]:
evol3 = alt.Chart(alls).mark_line(color='#1f77b4', point=True).encode(
    x=alt.X('year:N', scale=alt.Scale(zero=False), axis=alt.Axis(grid=True)),
    y=alt.Y('score:Q', scale=alt.Scale(zero=False)),
    tooltip=['Country Name:N','year:N','score:Q']
).add_selection(
    indicatorSelect,
    countrySelect3
).transform_filter(
    countrySelect3 & indicatorSelect
).properties(
    width=300,
    height=200
)

text3 = alt.Chart(alls).mark_text(dy=-120, size=18).encode(
    text='Country Name:N'
).transform_filter(
    countrySelect3 & indicatorSelect
)

Finally, we arrange all charts we created in a suitable set-up, with its corresponding text ('title') charts and with a last configuration of the line chart points to red color.

In [ ]:
final = ((bars | gdp_indices+text) & (evol1+text1 | evol2+text2 | evol3+text3)).configure_point(color = '#ba2207')

The result we get is:

In [ ]:
final

alt.VConcatChart(...)

In order to access it quickly and embed the data in a single file, we save all in a HTML file.

In [ ]:
final.save('final.html')